## Milestone 1 - Plot the World

In [ ]:
"""
**************************************************************************
*
* Author: Cooper Wolf
* Email: ctwolf1014@my.msutexas.edu
* Label: P01
* Title: Missile Geometry
* Course: CMPS 5993
* Semester: Spring 2026
*
* Description:
* This noteboook visualizes the geometry of a missile's range using Folium and GeoPandas.
* It displays a world map with country boundaries and marks the base location from which the missile is launched.
*
* 
*
* Files:
* Missile_Geometry.ipynb        : driver program
* countries.geo.json            : GeoJSON file containing world country boundaries
*
**************************************************************************
"""

In [ ]:
# Installing libraries
%pip install folium geopandas

In [1]:
# Importing libraries
import geopandas as gpd
import folium

# Reading the world data
world = gpd.read_file("countries.geo.json")

# Creating a map
m = folium.Map(location=[20,0], zoom_start=2)

In [2]:
# Converting GeoDataFrame to GeoJSON and adding to Folium
folium.GeoJson(world, name="World Countries").add_to(m)

In [3]:
# Adding base location as a point marker
base_lat = 38.89767
base_lon = -77.03655

# Adding a marker for the base location
folium.Marker(location=[base_lat, base_lon], popup="Base Location", icon=folium.Icon(color="red", icon="info-sign")).add_to(m)

In [ ]:
m